<a href="https://colab.research.google.com/github/karlacuv/MCD_Procesamiento/blob/main/Tarea7_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Learning
### Abril Grisel Guevara Cedillo - 1419239
### Kin Michelle Neváres Ríos - 1671272
### Karla Cureño Vega - 2085376

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/My Drive/Files/MCD/Procesamiento y Clasificación de Datos/Base Miniproyecto 2/archive.zip'

Archive:  /content/drive/My Drive/Files/MCD/Procesamiento y Clasificación de Datos/Base Miniproyecto 2/archive.zip
replace test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#vgg16 model
from keras.applications.vgg16 import VGG16
# load model
model = VGG16()
# summarize the model
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

Como primer paso se importa la libreria glob la cual nos ayudara a importar las imagenes por categoria, y modificarlas para poder usar el transfer learning.

Como regla inicial para todas las arquitecturas, el tamano de las imagenes debe de ser 224x224.

In [ ]:
# For training set only
import glob
import tensorflow as tf
import numpy as np

angry = glob.glob('train/angry/*.*')
disgust = glob.glob('train/disgust/*.*')
fear = glob.glob('train/fear/*.*')
happy = glob.glob('train/happy/*.*')
neutral = glob.glob('train/neutral/*.*')
sad = glob.glob('train/sad/*.*')
surprise = glob.glob('train/surprise/*.*')
data = []
labels = []
for i in angry:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Angry')
for i in disgust:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Disgust')
for i in fear:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Fear')
for i in happy:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Happy')
for i in neutral:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Neutral')
for i in sad:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Sad')
for i in surprise:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Surprise')
train_data = np.array(data)
train_labels = np.array(labels)

In [ ]:
train_data.shape

(28709, 48, 48)

In [ ]:
# For test set

angry = glob.glob('test/angry/*.*')
disgust = glob.glob('test/disgust/*.*')
fear = glob.glob('test/fear/*.*')
happy = glob.glob('test/happy/*.*')
neutral = glob.glob('test/neutral/*.*')
sad = glob.glob('test/sad/*.*')
surprise = glob.glob('test/surprise/*.*')
data = []
labels = []
for i in angry:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Angry')
for i in disgust:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Disgust')
for i in fear:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Fear')
for i in happy:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Happy')
for i in neutral:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Neutral')
for i in sad:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Sad')
for i in surprise:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='grayscale')
    image=np.array(image)
    data.append(image)
    labels.append('Surprise')
test_data = np.array(data)
test_labels = np.array(labels)

In [ ]:
test_data.shape

(7178, 48, 48)

Normalizamos los datos y hacemos una transformación a las etiquetas que pusimos anteriormente.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

X_train = train_data.astype('float32')
X_test = test_data.astype('float32')
X_train /= 255
X_test /= 255

lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(train_labels))
y_test = np_utils.to_categorical(lb.fit_transform(test_labels))

In [ ]:
vgg_model = VGG16(weights=None,include_top=False, input_shape=(48, 48, 1))

for layer in vgg_model.layers:
  layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(vgg_model.layers):
    print(i, layer.name, layer.trainable)

0 input_6 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False


In [ ]:
from keras.layers import Flatten, Dense, Dropout
from keras import Model

x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = Dense(7, activation='softmax')(x) # Softmax for multiclass
transfer_model = Model(inputs=vgg_model.input, outputs=x)

In [ ]:
from keras import optimizers

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
learning_rate= 5e-5
transfer_model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=["accuracy"])
history = transfer_model.fit(X_train, y_train, batch_size = 100, callbacks=[callback], epochs=50, validation_data=(X_test,y_test))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
288/288 [==============================] - 51s 175ms/step - loss: 1.8405 - accuracy: 0.2513 - val_loss: 1.8418 - val_accuracy: 0.2471
Epoch 2/50
288/288 [==============================] - 40s 138ms/step - loss: 1.8390 - accuracy: 0.2513 - val_loss: 1.8404 - val_accuracy: 0.2471
Epoch 3/50
288/288 [==============================] - 40s 139ms/step - loss: 1.8376 - accuracy: 0.2513 - val_loss: 1.8392 - val_accuracy: 0.2471
Epoch 4/50
288/288 [==============================] - 39s 137ms/step - loss: 1.8363 - accuracy: 0.2513 - val_loss: 1.8380 - val_accuracy: 0.2471
Epoch 5/50
288/288 [==============================] - 50s 175ms/step - loss: 1.8351 - accuracy: 0.2513 - val_loss: 1.8369 - val_accuracy: 0.2471
Epoch 6/50
288/288 [==============================] - 50s 175ms/step - loss: 1.8340 - accuracy: 0.2513 - val_loss: 1.8358 - val_accuracy: 0.2471
Epoch 7/50
288/288 [==============================] - 41s 142ms/step - loss: 1.8329 - accuracy: 0.2513 - val_loss: 1.8348 - val_ac

In [ ]:
transfer_model.save('my_model_BS100.h5')

Abril: Lectura de todas las imágenes con formato RGB, los pesos de la red se inicializan con ImageNet. La red fue entrenada con un batch_size de 300 y 30 epochs. El accuracy final en el set de validación fue de 0.2471.

Karla: Lectura de todas las imágenes con formato escala de grises, los pesos de la red se inicializan de manera aleatoria. La red fue entrenada con un batch_size de 100 y 50 epochs. El accuracy final en el set de validación fue de 0.2471.

Kin: Lectura de una muestra de las imágenes para optimizar tiempo de ejcución con formato escala de grises, los pesos de la red se inicializan de manera aleatoria. La red fue entrenada con un batch_size de 1 y 30 epochs. El accuracy final en el set de validación fue de 0.2369.

### Conclusión

Debido a que las imágenes de nuestro dataset están en escala de grises y la red preentrenada está optimizada para imágenes en RGB, esta red no es adecuada para este set de datos, pues al leer nuestras imágenes con formato RGB, éstas siguen estando en escala de grises.

Con la modificación de los pesos para que empezaran de manera aleatoria en vez de con los pesos de ImageNet, hace que sea muy díficil para la red encontrar los pesos adecuados, por lo que la precisión no mejora.

Además, el tamaño de las imágenes en el dataset es muy pequeño por lo que tener una red tan densa hace que se pierda mucha información de la imagen en cada capa de la red.

Al extraer una muestra de todas las imágenes, el tiempo de ejecución mejora, sin embargo, el "accuracy" empeora un poco.